# Exploring tensor decompositions in pyTorch
Simon Aertssen
12/03/2020

In [14]:
import torch
import sys
print(sys.version)

3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


## 1. Useful functions and products:


The $\textbf{Matrix Kronecker product}$ is defined as

\begin{align*} \boldsymbol{P}^{\:\mathrm{I} \times \mathrm{J}} \otimes \boldsymbol{Q}^{\:\mathrm{K} \times \mathrm{L}}=\boldsymbol{R}^{\:\mathrm{IK} \times \mathrm{JL}}, \text { such that } r_{k+K(i-1), \: l+L(j-1)}=p_{i j} q_{k l} \end{align*}

Naive: use loops. Smart: reshape P and Q so that their dimensions do not overlap, then the elementwise or Hadamard product gives the desired result.

See [Wikipedia](https://en.wikipedia.org/wiki/Kronecker_product) for the examples.


In [139]:
def KronProd2D(P, Q):
    # Register dimensions.
    I, J = P.shape
    K, L = Q.shape
    
    # Adjust dimensions of P and Q to perform smart multiplication:
    # interweave the dimensions containing values and perform elementwise multiplication.
    P = P.view(I, 1, J, 1)
    Q = Q.view(1, K, 1, L)
    
    R = P * Q
    return R.view(I*K, J*L)

def KronProd(P, Q):
    # This should work for higher order tensors.
    # Register and check dimensions.
    pshape = P.shape
    qshape = Q.shape
    if len(pshape) != len(qshape):
        raise ValueError('Matrices should be of the same order: ' + str(list(pshape)) + ' != ' + str(list(qshape)))
    
    # Adjust dimensions of P and Q to perform smart multiplication:
    # interweave the dimensions containing values and perform elementwise multiplication.
    # Start with a list of ones and set dimensions as every even or uneven index.
    pindices = [1]*2*len(pshape)    
    pindices[::2] = pshape
    qindices = [1]*2*len(qshape)
    qindices[1::2] = qshape
        
    P = P.view(pindices)
    Q = Q.view(qindices)
    
    R = P * Q
    rshape = [p*q for p, q in zip(pshape,qshape)]
    return R.view(rshape)

Example:

In [140]:
P = torch.tensor([[1, -4, 7], [-2, 3, 3]])
Q = torch.tensor([[8, -9, -6, 5], [1, -3, -4, 7], [2, 8, -8, -3], [1, 2, -5, -1]])
R = KronProd(P, Q)
assert torch.all(R.eq(KronProd2D(P, Q)))

The $\textbf{Kahtri-Rhao product}$ is defined as

\begin{align*}
\boldsymbol{P}^{\:\mathrm{I} \times \mathrm{J}}|\otimes| \boldsymbol{Q}^{\mathrm{K} \times \mathrm{J}}=\boldsymbol{P}^{\:\mathrm{I} \times \mathrm{J}} \odot \boldsymbol{Q}^{\mathrm{K} \times \mathrm{J}}=\boldsymbol{R}^{\:\mathrm{IK} \times \mathrm{J}}, \text { such that } r_{k+K(i-1), j}=p_{i j} q_{k j}
\end{align*}

This can be seen as a column-wise kronecker product.

In [207]:
def KathRaoProd(P, Q):
    # Register and check dimensions
    pshape = P.shape
    qshape = Q.shape
    J = pshape[-1]
    if J != qshape[-1]:
        raise ValueError('Matrices should have the same number of columns: ' + str(J) + ' != ' + str(qshape[-1]))
    
    # Make R an empty tensor
    rshape = [p*q for p, q in zip(pshape,qshape)]
    rshape[-1] = J
    R = torch.zeros(rshape)
    for j in range(J):        
        R[:,j] = KronProd(P[j], Q[j])
        
    .narrow(0, 0, 2)
        
    # Tried with slicing but did not seem to work:
    # column_indices = list(range(J)) or slice(0,J)
    # R[column_indices] = KronProd(P[column_indices], Q[column_indices])
    return R


Example:

In [210]:
P = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
Q = torch.tensor([[1, 4, 7], [2, 5, 8], [3, 6, 9]])
R = KathRaoProd(P, Q)
#print(R)